# NN Playground

This notebook will be testing out variations on different neural networks, utilizing various forms of cells/nodes.

Cell types include:
-Backfed input Cell
-Input Cell
-Noisy Input Cell

# Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Base NN Class

Creating a BaseNN class intended to use inheritance in later implementations of different NN's when abstracting the base class to make specialized classes.

In [2]:
# Base class for neural networks
class BaseNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BaseNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

    def forward(self, x):
        raise NotImplementedError("forward method must be implemented in derived classes")

In [3]:
# Instantiate the neural networks
input_size = 2
hidden_size = 3

# Basic Classes

## Perceptron (P)

In [4]:
class Perceptron:
    def __init__(self, input_size):
        # Initialize weights and bias randomly
        self.weights = np.random.rand(input_size)
        self.bias = np.random.rand()

    def activate(self, x):
        # Simple step function as activation
        return 1 if x > 0 else 0

    def forward(self, inputs):
        # Calculate the weighted sum of inputs
        weighted_sum = np.dot(inputs, self.weights) + self.bias

        # Apply the activation function
        output = self.activate(weighted_sum)

        return output

In [5]:
# Example Usage
if __name__ == "__main__":
    # Create a perceptron with 2 input cells
    perceptron = Perceptron(input_size=2)

    # Example input
    input_data = np.array([0.5, 0.8])

    # Get the output from the perceptron
    output = perceptron.forward(input_data)

    print(f"Input: {input_data}")
    print(f"Output: {output}")

Input: [0.5 0.8]
Output: 1


## Feed Forward (FF)

In [6]:
class FeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FeedforwardNN, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layer = nn.Linear(hidden_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, 1)  # Single output neuron

    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        x = torch.relu(self.hidden_layer(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# Instantiate the neural network
input_size = 2  # Number of input features 
hidden_size = 3  # Number of neurons in the hidden layers
model = FeedforwardNN(input_size, hidden_size)

In [7]:
# Define a sample input
sample_input = torch.tensor([[0.5, 0.3]])  # Example Data

# Forward pass to get the output
output = model(sample_input)

# Print the model architecture and output
print(model)
print("Output:", output.item())

FeedforwardNN(
  (input_layer): Linear(in_features=2, out_features=3, bias=True)
  (hidden_layer): Linear(in_features=3, out_features=3, bias=True)
  (output_layer): Linear(in_features=3, out_features=1, bias=True)
)
Output: 0.47794511914253235


# Radial Basis Network (RBF)

In [8]:
class RadialBasisFunction:
    def __init__(self, input_size, num_centers):
        # Initialize centers and width parameters randomly
        self.centers = np.random.rand(num_centers, input_size)
        self.width = np.random.rand()
        self.weights = np.random.rand(num_centers)
    
    def gaussian(self, x, center, width):
        # Gaussian activation function
        return np.exp(-np.sum((x - center)**2) / (2 * width**2))
    
    def forward(self, inputs):
        # Calculate the activation for each center
        activations = np.array([self.gaussian(inputs, center, self.width) for center in self.centers])
        
        # Calculate the weighted sum of activations
        weighted_sum = np.dot(activations, self.weights)
        
        # Apply a threshold for binary output
        output = 1 if weighted_sum > 0.5 else 0
        
        return output

In [9]:
# Example Usage
if __name__ == "__main__":
    # Create an RBF network with 2 input cells and 3 centers
    rbf_network = RadialBasisFunction(input_size=2, num_centers=3)
    
    # Example input
    input_data = np.array([0.5, 0.8])
    
    # Get the output from the RBF network
    output = rbf_network.forward(input_data)
    
    print(f"Input: {input_data}")
    print(f"Output: {output}")

Input: [0.5 0.8]
Output: 1


# Recurrent Neural Network (RNN)

In [10]:
class SimpleRNN(BaseNN):
    def __init__(self, input_size, hidden_size):
        super(SimpleRNN, self).__init__(input_size, hidden_size, output_size=1)
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.recurrent_layer = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, self.output_size)

    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        h_t, _ = self.recurrent_layer(x)
        output = torch.sigmoid(self.output_layer(h_t[:, -1, :]))  # Taking the output from the last time step
        return output

## Comparing FF & RNN

In [11]:
# Instantiate the models
feedforward_model = FeedforwardNN(input_size, hidden_size)
simple_rnn_model = SimpleRNN(input_size, hidden_size)

# Forward pass for the feedforward model
sample_input = torch.tensor([[0.5, 0.3]])
output_feedforward = feedforward_model(sample_input)

# Forward pass for the simple RNN model
sample_input_rnn = torch.rand((1, 4, input_size))
output_rnn = simple_rnn_model(sample_input_rnn)

# Print the model architectures and outputs
print("Feedforward Model:")
print(feedforward_model)
print("Output:", output_feedforward.item())

print("\nSimple RNN Model:")
print(simple_rnn_model)
print("Output:", output_rnn.item())

Feedforward Model:
FeedforwardNN(
  (input_layer): Linear(in_features=2, out_features=3, bias=True)
  (hidden_layer): Linear(in_features=3, out_features=3, bias=True)
  (output_layer): Linear(in_features=3, out_features=1, bias=True)
)
Output: 0.729799747467041

Simple RNN Model:
SimpleRNN(
  (input_layer): Linear(in_features=2, out_features=3, bias=True)
  (recurrent_layer): RNN(3, 3, batch_first=True)
  (output_layer): Linear(in_features=3, out_features=1, bias=True)
)
Output: 0.7493574023246765


# Deep Feed Forward (DFF)

In [12]:
# Deep Feed Forward Neural Network
class DeepFeedforwardNN(BaseNN):
    def __init__(self, input_size, hidden_size, output_size):
        super(DeepFeedforwardNN, self).__init__(input_size, hidden_size, output_size)
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_size, hidden_size) for _ in range(2)  # Two hidden layers with 4 nodes each
        ])
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

In [13]:
# Instantiate the deep feedforward neural network
input_size = 3  # Number of input features 
hidden_size = 4  # Number of nodes in each hidden layer
output_size = 2  # Number of output nodes
deep_feedforward_model = DeepFeedforwardNN(input_size, hidden_size, output_size)

# Define a sample input
sample_input = torch.tensor([[0.5, 0.3, 0.8]])  # Example Data

# Forward pass to get the output
output_deep_feedforward = deep_feedforward_model(sample_input)

# Print the model architecture and output
print(deep_feedforward_model)
print("Output:", output_deep_feedforward)

DeepFeedforwardNN(
  (input_layer): Linear(in_features=3, out_features=4, bias=True)
  (hidden_layers): ModuleList(
    (0-1): 2 x Linear(in_features=4, out_features=4, bias=True)
  )
  (output_layer): Linear(in_features=4, out_features=2, bias=True)
)
Output: tensor([[0.5660, 0.3460]], grad_fn=<SigmoidBackward0>)


# Long Short Term Memory (LSTM)

In [14]:
# LSTM Neural Network
class LSTMNN(BaseNN):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMNN, self).__init__(input_size, hidden_size, output_size)
        self.lstm_layer = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (h_t, c_t) = self.lstm_layer(x)
        output = torch.sigmoid(self.output_layer(h_t[-1, :, :]))  # Taking the output from the last time step
        return output

In [15]:
# Instantiate the LSTM neural network
input_size = 3  # Number of input features 
hidden_size = 3  # Number of memory cells
output_size = 4  # Number of output nodes
lstm_model = LSTMNN(input_size, hidden_size, output_size)

# Define a sample input
sample_input = torch.rand((1, 4, input_size))  # Example Data

# Forward pass to get the output
output_lstm = lstm_model(sample_input)

# Print the model architecture and output
print(lstm_model)
print("Output:", output_lstm)

LSTMNN(
  (lstm_layer): LSTM(3, 3, batch_first=True)
  (output_layer): Linear(in_features=3, out_features=4, bias=True)
)
Output: tensor([[0.3917, 0.5766, 0.5157, 0.4375]], grad_fn=<SigmoidBackward0>)


# Gated Recurrent Unit (GRU)

In [16]:
class GRUNN(BaseNN):
    def __init__(self, input_size, hidden_size, output_size=1):
        super(GRUNN, self).__init__(input_size, hidden_size, output_size)
        self.gru_layer = nn.GRU(input_size, hidden_size, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, self.output_size)

    def forward(self, x):
        h_t, _ = self.gru_layer(x)
        output = torch.sigmoid(self.output_layer(h_t[:, -1, :]))  # Taking the output from the last time step
        return output

In [17]:
# Instantiate the GRU neural network
input_size = 3  # Number of input features 
hidden_size = 3  # Number of memory cells
output_size = 4  # Number of output nodes
gru_model = GRUNN(input_size, hidden_size, output_size)

# Define a sample input
sample_input = torch.rand((1, 4, input_size))  # Example Data

# Forward pass to get the output
output_gru = gru_model(sample_input)

# Print the model architecture and output
print(gru_model)
print("Output:", output_gru)

GRUNN(
  (gru_layer): GRU(3, 3, batch_first=True)
  (output_layer): Linear(in_features=3, out_features=4, bias=True)
)
Output: tensor([[0.4354, 0.5022, 0.5707, 0.5334]], grad_fn=<SigmoidBackward0>)


# Auto Encoder (AE)

AE designed for unsupervised learning & Data compression.

Learns compact representation of input data.

Used for data denoising, dimensionality reduction, feature learning.

Versitile building block in utilizing NN's.

In [18]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Linear(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = torch.sigmoid(self.decoder(encoded))
        return decoded

In [19]:
# Instantiate the autoencoder
input_size = 10  # Number of input features
hidden_size = 5  # Number of hidden nodes (compressed representation)
autoencoder = Autoencoder(input_size, hidden_size)

# Define a sample input
sample_input = torch.rand((1, input_size))

# Forward pass to get the reconstructed output
output_autoencoder = autoencoder(sample_input)

# Print the model architecture and output
print(autoencoder)
print("Output:", output_autoencoder)

Autoencoder(
  (encoder): Linear(in_features=10, out_features=5, bias=True)
  (decoder): Linear(in_features=5, out_features=10, bias=True)
)
Output: tensor([[0.4270, 0.5819, 0.6050, 0.5539, 0.4236, 0.5984, 0.6222, 0.5810, 0.5461,
         0.4906]], grad_fn=<SigmoidBackward0>)


# Variational AE (VAE)

VAE adds probabilistic approach to encoding. Generates deterministic encoding & probabilistic distribution over possible encodings. 

This enables model to generate new data points by sampling from the learned distribution.

VAE good tool for data generation tasks.

In [20]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(VariationalAutoencoder, self).__init__()

        # Encoder layers
        self.encoder_fc1 = nn.Linear(input_size, hidden_size)
        self.encoder_fc2_mean = nn.Linear(hidden_size, hidden_size)
        self.encoder_fc2_logvar = nn.Linear(hidden_size, hidden_size)

        # Decoder layers
        self.decoder_fc1 = nn.Linear(hidden_size, input_size)
        self.decoder_fc2 = nn.Linear(input_size, input_size)

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def forward(self, x):
        # Encoder
        x = torch.relu(self.encoder_fc1(x))
        mean = self.encoder_fc2_mean(x)
        logvar = self.encoder_fc2_logvar(x)

        # Reparameterization trick
        z = self.reparameterize(mean, logvar)

        # Decoder
        x_hat = torch.relu(self.decoder_fc1(z))
        x_hat = torch.sigmoid(self.decoder_fc2(x_hat))

        return x_hat, mean, logvar

In [21]:
# Instantiate the variational autoencoder
input_size = 4  # Number of input features
hidden_size = 4  # Number of hidden nodes in probabilistic layer
vae = VariationalAutoencoder(input_size, hidden_size)

# Define a sample input
sample_input = torch.rand((1, input_size))

# Forward pass to get the reconstructed output and latent variables
output_vae, mean, logvar = vae(sample_input)

# Print the model architecture and output
print(vae)
print("Output:", output_vae)
print("Mean:", mean)
print("Log Variance:", logvar)

VariationalAutoencoder(
  (encoder_fc1): Linear(in_features=4, out_features=4, bias=True)
  (encoder_fc2_mean): Linear(in_features=4, out_features=4, bias=True)
  (encoder_fc2_logvar): Linear(in_features=4, out_features=4, bias=True)
  (decoder_fc1): Linear(in_features=4, out_features=4, bias=True)
  (decoder_fc2): Linear(in_features=4, out_features=4, bias=True)
)
Output: tensor([[0.4220, 0.4546, 0.5274, 0.3855]], grad_fn=<SigmoidBackward0>)
Mean: tensor([[ 0.1978, -0.5265, -0.4358,  0.5759]], grad_fn=<AddmmBackward0>)
Log Variance: tensor([[0.7016, 0.8123, 0.0279, 0.1598]], grad_fn=<AddmmBackward0>)


# Denoising Auto Encoder

DAE corrupts input data w/ noise. Therefore DAEs are forced to learn most robust & meaningful features.
Results in resiliance against noisy inputs.

In [22]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(DenoisingAutoencoder, self).__init__()

        # Encoder layers
        self.encoder_fc1 = nn.Linear(input_size, hidden_size)

        # Decoder layers
        self.decoder_fc1 = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        # Encoder
        x = torch.relu(self.encoder_fc1(x))

        # Decoder
        x_hat = torch.sigmoid(self.decoder_fc1(x))

        return x_hat

In [23]:
# Instantiate the denoising autoencoder
input_size = 4  # Number of input features
hidden_size = 4  # Number of hidden nodes
dae = DenoisingAutoencoder(input_size, hidden_size)

# Define a sample input (noisy data)
noisy_input = torch.rand((1, input_size))  # Example Noisy Data

# Forward pass to get the reconstructed output
output_dae = dae(noisy_input)

# Print the model architecture and output
print(dae)
print("Noisy Input:", noisy_input)
print("Reconstructed Output:", output_dae)

DenoisingAutoencoder(
  (encoder_fc1): Linear(in_features=4, out_features=4, bias=True)
  (decoder_fc1): Linear(in_features=4, out_features=4, bias=True)
)
Noisy Input: tensor([[0.0886, 0.3265, 0.2430, 0.4412]])
Reconstructed Output: tensor([[0.4467, 0.4789, 0.6245, 0.4430]], grad_fn=<SigmoidBackward0>)


# Sparse Auto Encoder

SAE involves additional constraint in the training process to encourage network to learn sparse representations w/ goal of learn more robust & efficent representations by promoting small number of neurons to be activate at a time. 

In [24]:
class SparseAutoencoder(BaseNN):
    def __init__(self, input_size, hidden_size, sparsity_target=0.1, sparsity_weight=0.2):
        super(SparseAutoencoder, self).__init__(input_size, hidden_size, output_size=input_size)

        # Encoder layers
        self.encoder_fc1 = nn.Linear(input_size, hidden_size)

        # Decoder layers
        self.decoder_fc1 = nn.Linear(hidden_size, input_size)

        # Sparsity parameters
        self.sparsity_target = sparsity_target
        self.sparsity_weight = sparsity_weight
        self.relu = nn.ReLU()

    def forward(self, x):
        # Encoder
        encoded = self.encoder_fc1(x)
        encoded = self.relu(encoded)

        # Decoder
        decoded = torch.sigmoid(self.decoder_fc1(encoded))

        return decoded, encoded

    def loss_function(self, x, x_hat, encoded):
        # Reconstruction loss
        reconstruction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction='mean')

        # Sparsity loss
        sparsity_loss = torch.sum(self.kl_divergence(self.sparsity_target, encoded))

        # Total loss
        total_loss = reconstruction_loss + self.sparsity_weight * sparsity_loss

        return total_loss

    def kl_divergence(self, target, activations):
        # KL Divergence to enforce sparsity
        p = torch.mean(activations, dim=0)  # Average activation over the dataset
        return target * torch.log(target / p) + (1 - target) * torch.log((1 - target) / (1 - p))

In [25]:
# Instantiate the sparse autoencoder
input_size = 5  # Number of input features
hidden_size = 3  # Number of hidden nodes
sae = SparseAutoencoder(input_size, hidden_size)

# Define a sample input
sample_input = torch.rand((1, input_size))  # Example Data

# Forward pass to get the reconstructed output and encoded representation
output_sae, encoded_sae = sae(sample_input)

# Calculate the loss
loss_sae = sae.loss_function(sample_input, output_sae, encoded_sae)

# Print the model architecture, output, and loss
print(sae)
print("Input:", sample_input)
print("Reconstructed Output:", output_sae)
print("Encoded Representation:", encoded_sae)
print("Loss:", loss_sae.item())

SparseAutoencoder(
  (encoder_fc1): Linear(in_features=5, out_features=3, bias=True)
  (decoder_fc1): Linear(in_features=3, out_features=5, bias=True)
  (relu): ReLU()
)
Input: tensor([[0.6333, 0.5426, 0.7588, 0.3070, 0.5324]])
Reconstructed Output: tensor([[0.5462, 0.4714, 0.5358, 0.5153, 0.4913]], grad_fn=<SigmoidBackward0>)
Encoded Representation: tensor([[0.8522, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)
Loss: inf


# Markov Chain (MC)

In [26]:
class MarkovChainNN(BaseNN):
    def __init__(self, input_size, hidden_size):
        super(MarkovChainNN, self).__init__(input_size, hidden_size, output_size=input_size)
        self.transition_matrix = nn.Parameter(torch.randn(input_size, hidden_size))
        self.output_layer = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        # Apply a simple linear transformation based on the transition matrix
        x = torch.matmul(x, self.transition_matrix)
        # Apply a linear layer to get the final output
        output = self.output_layer(x)
        # You might want to apply some non-linearity here based on your specific needs
        # For example, you can use torch.relu(output) or torch.sigmoid(output) depending on the task
        return output

# Instantiate the Markov Chain neural network
input_size = 4  # Number of input features 
hidden_size = 8  # Number of hidden states
markov_chain_model = MarkovChainNN(input_size, hidden_size)

# Define a sample input
sample_input = torch.rand((1, input_size))  # Example Data

# Forward pass to get the output
output_markov_chain = markov_chain_model(sample_input)

In [27]:
# Print the model architecture and output
print(markov_chain_model)
print("Output:", output_markov_chain)

MarkovChainNN(
  (output_layer): Linear(in_features=8, out_features=4, bias=True)
)
Output: tensor([[ 0.0210, -0.0391, -0.7962, -0.0839]], grad_fn=<AddmmBackward0>)


# Hopfield Network

In [28]:
class HopfieldNetwork(BaseNN):
    def __init__(self, input_size):
        super(HopfieldNetwork, self).__init__(input_size, hidden_size=None, output_size=input_size)
        
        # Weight matrix for the Hopfield Network
        self.weights = nn.Parameter(torch.zeros((input_size, input_size), dtype=torch.float))

    def forward(self, x):
        # Apply the Hopfield Network dynamics
        y = torch.sign(x @ self.weights).long()  # Convert to torch.long after applying sign
        return y


In [29]:
# Example usage
input_size = 5  # Change this based on your needs
hopfield_model = HopfieldNetwork(input_size)

# Define a sample input pattern (1 or -1)
sample_input = torch.tensor([[1, -1, 1, -1, 1]], dtype=torch.float)  # Change the data type to torch.float

# Forward pass to retrieve the output
output_hopfield = hopfield_model(sample_input)

# Print the model architecture and output
print(hopfield_model)
print("Output:", output_hopfield.numpy())

HopfieldNetwork()
Output: [[0 0 0 0 0]]


# Boltzmann Machine (BM)

BM is a stochastic RNN designed to find a probability distribution over its set of binary-valued patterns. 

*Main Objective* is to learn the joint probablity distribution of its training data.

Unique Features:
-visible & hidden units forming bipartite graph
-connects between units have weights & model learns weights during training
-stochastic update process for both training & inference

Common Uses:
-unsupervised learning tasks like feature learning, dimensionality reduction, density estimation.

In [30]:
class BoltzmannMachine(nn.Module):
    def __init__(self, num_visible, num_hidden):
        super(BoltzmannMachine, self).__init__()
        self.num_visible = num_visible
        self.num_hidden = num_hidden

        # Define the parameters (weights and biases)
        self.weights = nn.Parameter(torch.randn(num_visible, num_hidden))
        self.visible_bias = nn.Parameter(torch.randn(num_visible))
        self.hidden_bias = nn.Parameter(torch.randn(num_hidden))

    def forward(self, visible_states):
        # Ensure visible_states has the correct dimensions (batch_size x num_visible)
        if visible_states.dim() == 1:
            visible_states = visible_states.view(1, -1)

        # Compute the hidden probabilities given visible states
        hidden_probabilities = F.sigmoid(F.linear(visible_states, self.weights.t(), self.hidden_bias))

        # Sample hidden states from the computed probabilities
        hidden_states = torch.bernoulli(hidden_probabilities)

        # Compute the visible probabilities given the sampled hidden states
        visible_probabilities = F.sigmoid(F.linear(hidden_states, self.weights, self.visible_bias))

        # Sample visible states from the computed probabilities
        visible_states = torch.bernoulli(visible_probabilities)

        return visible_states, hidden_states

In [31]:
# Example usage
num_visible = 5
num_hidden = 3

boltzmann_machine = BoltzmannMachine(num_visible, num_hidden)

# Define a sample visible state (binary values)
sample_visible_state = torch.tensor([1, 0, 1, 0, 1.], dtype=torch.float)

# Perform a Gibbs sampling step
sampled_visible, sampled_hidden = boltzmann_machine(sample_visible_state)

# Print the model architecture and sampled states
print(boltzmann_machine)
print("Sampled Visible State:", sampled_visible.detach().numpy())
print("Sampled Hidden State:", sampled_hidden.detach().numpy())

BoltzmannMachine()
Sampled Visible State: [[0. 0. 1. 1. 1.]]
Sampled Hidden State: [[0. 1. 1.]]


# Restricted Boltzmann Machine (RBM)

Differences from normal Boltzmann Machine:

    -No connections between units within same layer (no hidden-hidden or visible-visible connections)
    
    -Bipartite graph with one layer of visible units and one layer of hidden units
    
Objective: 

    -RBM objective with changes for potentially more effective feature learning

Unique Features:

    -Widely used for feature learning & are building blocks in deep learning architectures.
    
    -Efficent training, Contrastive Divergence (CD) is often used for training RBMs
Common Uses:

    -Feature Learning; pre-training deep NN
    
    -Collaborative filtering, topic modeling, other unsupervised learning tasks.

In [32]:
class RBM(BaseNN):
    def __init__(self, visible_size, hidden_size):
        super(RBM, self).__init__(visible_size, hidden_size, None)
        self.weights = nn.Parameter(torch.randn(visible_size, hidden_size))
        self.visible_bias = nn.Parameter(torch.zeros(visible_size))
        self.hidden_bias = nn.Parameter(torch.zeros(hidden_size))

    def forward(self, x):
        hidden_prob = F.sigmoid(F.linear(x, self.weights.t(), self.hidden_bias))
        hidden_state = torch.bernoulli(hidden_prob)
        reconstructed_prob = F.sigmoid(F.linear(hidden_state, self.weights, self.visible_bias))
        return hidden_state, reconstructed_prob

In [33]:
# Example usage
visible_size = 5
hidden_size = 3

# Create an RBM model
rbm_model = RBM(visible_size, hidden_size)

# Define a sample visible state (binary values)
sample_visible_state = torch.tensor([[1, 0, 1, 0, 1.]], dtype=torch.float)

# Forward pass to get the hidden states
hidden_states, _ = rbm_model(sample_visible_state)

# Print the model architecture and hidden states
print(rbm_model)
print("Sampled Visible State:", sample_visible_state.detach().numpy())
print("Hidden States:", hidden_states.detach().numpy())

RBM()
Sampled Visible State: [[1. 0. 1. 0. 1.]]
Hidden States: [[0. 1. 1.]]


# Deep Belief Network (DBN)

Objective:

    -Unsupervised learning & feature learning
    -Model complex hierarchical representations of data
Unique Features:
    
    -Multiple layer of stochastic, latent variables (usually binary)
    -Stack of Restricted Boltzmann Machines (See super in the DBN class init for implementing via making DBN layers)
    - Uses a layer-wise pre-training approach followed by fine-tuning using backprop
    
Common Uses:
    
    - Feature Learning
    - Generative tasks (new samples from learned distribution)

In [34]:
class DBN(BaseNN):
    def __init__(self, visible_size, hidden_sizes):
        super(DBN, self).__init__(visible_size, None, None)
        self.rbm_layers = nn.ModuleList([RBM(visible_size, hidden_size) for hidden_size in hidden_sizes])

    def forward(self, x):
        # Forward pass through each RBM layer
        for rbm_layer in self.rbm_layers:
            x, _ = rbm_layer(x)
        return x

In [35]:
# Example usage
visible_size = 5
hidden_sizes = [5, 1]

# Create a Deep Belief Network
dbn_model = DBN(visible_size, hidden_sizes)

# Define a sample input
sample_input = torch.rand((1, visible_size))

# Forward pass through the DBN
output_dbn = dbn_model(sample_input)

# Print the model architecture and output
print(dbn_model)
print("Output:", output_dbn.detach().numpy())

DBN(
  (rbm_layers): ModuleList(
    (0-1): 2 x RBM()
  )
)
Output: [[1.]]


# Deep Convolutional Network (DCN)

Objective:
    
    -Processing structured grid data like images
    -Excels @ capturing hierarchical spatial patterns

Unique Features:
    
    -Uses convolutional layers w/ learnable filters that capture local patterns
    -Typically includes pooling layers to reduce spatial dimensions & increase computational efficency
    -Uses shared weights in convolutional layers for translation invariance
    
Common uses:
    
    -Image classification
    -Feature learning in spatial data (hierarchical representations of spatial features)
    -Transfer learning (pre-trained CNNs on large datasets often fine-tuned for specific tasks)

In [36]:
class DeepCNN(BaseNN):
    def __init__(self, input_channels, num_classes, image_size):
        hidden_size = 64

        super(DeepCNN, self).__init__(input_size=image_size, hidden_size=hidden_size, output_size=num_classes)

        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * (image_size // 4) * (image_size // 4), hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * (self.input_size // 4) * (self.input_size // 4))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [37]:
# Example usage
input_channels = 1  # Grayscale images
num_classes = 10
image_size = 28

deep_cnn_model = DeepCNN(input_channels, num_classes, image_size)

sample_image = torch.rand((1, input_channels, image_size, image_size))

output_scores = deep_cnn_model(sample_image)

print(deep_cnn_model)
print("Output Scores:", output_scores.detach().numpy())

DeepCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)
Output Scores: [[ 0.12570626  0.10115343  0.03376044  0.06880514 -0.01437923 -0.09773049
  -0.01460256  0.09239314  0.05630538 -0.08280865]]


# Deconvolutional Network (DN)

Objective:
    
    -Reconstruction & generation of struuctured data (especially images)
    -specialize in capturing hierarchical spatial patterns

Unique Features:
    
    -deconvolutional layers w/ learnable filters for reconstructing spatial patterns
    -usually include unpooling layers to increase spatial dimensions while maintaining computational efficency
    -shared weights in deconvolutional layers to introduce translation invariance during reconstruction
    
Common uses:
    
    -image reconstruction & generation
    -feature learning in spatial data w/ focus on capturing hierarchical spatial patterns
    -semantic segmentation in images
    -inverse problems (ex.image restoration)
    -transfer learning (pre-trained on larger dataset -> fine-tuned for specicific reconstruction task)

In [41]:
class DeepDeconvNet(BaseNN):
    def __init__(self, input_channels, output_channels, output_size):
        hidden_size = 64

        super(DeepDeconvNet, self).__init__(input_size=None, hidden_size=hidden_size, output_size=output_size)

        self.fc1 = nn.Linear(hidden_size, 64 * (output_size // 4) * (output_size // 4))
        self.deconv1 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv2 = nn.ConvTranspose2d(in_channels=32, out_channels=output_channels, kernel_size=3, stride=2, padding=1, output_padding=1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = x.view(-1, 64, (self.output_size // 4), (self.output_size // 4))
        x = F.relu(self.deconv1(x))
        x = F.sigmoid(self.deconv2(x))
        return x

notes on parameters above: 

    channels: in context of images, 1=grayscale, 3=RGB
    kernel_size: size of convolutional kernel-filter, size of local region considered for each convolutional operation
    stride: step-size
    padding: zero-padding addied to input of each side, helps maintain/adjust spatial dimensions
    output_size: shape of output data

In [42]:
# Example usage
input_channels = 1  # Grayscale images
output_channels = 3  # Number of channels in the output image (e.g., RGB)
output_size = 28

deep_deconv_model = DeepDeconvNet(input_channels, output_channels, output_size)

sample_latent_vector = torch.rand((1, deep_deconv_model.hidden_size))

output_image = deep_deconv_model(sample_latent_vector)

print(deep_deconv_model)
print("Output Image Shape:", output_image.shape)

DeepDeconvNet(
  (fc1): Linear(in_features=64, out_features=3136, bias=True)
  (deconv1): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
  (deconv2): ConvTranspose2d(32, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
)
Output Image Shape: torch.Size([1, 3, 28, 28])
